In [25]:
import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

import os

### Check GPU working

In [26]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2024-05-09 06:39:18.243238: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 06:39:18.243286: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17753491478140827450
 xla_global_id: -1]

### Read data from csv file

In [27]:
df = pd.read_csv(f'{os.getcwd()}/data/article_contents/environment_3.csv')
df.head()

,Unnamed: 0,0
0,0,Skip to contentHI Now DailyContestsJoin Our Te...
1,1,Just ListenCalendarEventsHonolulu PrideActivi...
2,2,"7, 2024 at 5:40 PM HST|Updated: 14 hours agoE..."
3,3,Environmentalists hope it will force the state...
4,4,Syngenta is one of several companies taking ad...


### Print news headline

In [30]:
headline = []
# 헤드라인의 값들을 리스트로 저장
headline.extend(list(df['0']))
headline[:]

['Skip to contentHI Now DailyContestsJoin Our TeamAdvertise with UsLiveNewsMaui WildfiresWeatherSunriseSportsPodcastsHI Now DailyEntertainmentActivitiesAbout UsHomeWatch LiveNewsLocal HeadlinesNational NewsHNN InvestigatesThis Is NowSubmit Your Photos!Latest NewscastsWeatherLatest ForecastsHNN Hurricane CenterSurf ReportTsunami CenterTrafficSunriseAs Seen On SunriseHealthier HawaiiWhats TrendingHowards Business ReportSmart Money MondayCelebrating Fil-Am HistoryHawaiian Word Of The DaySpecial ReportsSpotlight NowPodcastsHawaii News NowIsland BeatMuthaship with Stephanie LumThe Other Side of Paradise with Lynn KawanoHNN OvertimeHNN Off-AirTell Me A Story with Jim MendozaSportsSuper Bowl LVIIIBracket BustersWarrior NationScoringLiveStats & PredictionsHow to WatchEntertainmentNa Hoku Hanohano AwardsKeiki HulaCheap EatsPacific PulseKupuna PowerK5 MeleTeen TalkContestsHI Now DailyHI Now JapanTalk StoryCommunityMy Life',
 ' Just ListenCalendarEventsHonolulu PrideActivitiesAbout UsAnchors & Re

In [31]:
print('총 샘플의 개수 : {}'.format(len(headline)))

총 샘플의 개수 : 28


In [32]:
headline = [word for word in headline if word != "Unknown"]
print("노이즈값 제거 후 샘플의 개수 : {}".format(len(headline)))

노이즈값 제거 후 샘플의 개수 : 28


In [33]:
headline[:5]

['Skip to contentHI Now DailyContestsJoin Our TeamAdvertise with UsLiveNewsMaui WildfiresWeatherSunriseSportsPodcastsHI Now DailyEntertainmentActivitiesAbout UsHomeWatch LiveNewsLocal HeadlinesNational NewsHNN InvestigatesThis Is NowSubmit Your Photos!Latest NewscastsWeatherLatest ForecastsHNN Hurricane CenterSurf ReportTsunami CenterTrafficSunriseAs Seen On SunriseHealthier HawaiiWhats TrendingHowards Business ReportSmart Money MondayCelebrating Fil-Am HistoryHawaiian Word Of The DaySpecial ReportsSpotlight NowPodcastsHawaii News NowIsland BeatMuthaship with Stephanie LumThe Other Side of Paradise with Lynn KawanoHNN OvertimeHNN Off-AirTell Me A Story with Jim MendozaSportsSuper Bowl LVIIIBracket BustersWarrior NationScoringLiveStats & PredictionsHow to WatchEntertainmentNa Hoku Hanohano AwardsKeiki HulaCheap EatsPacific PulseKupuna PowerK5 MeleTeen TalkContestsHI Now DailyHI Now JapanTalk StoryCommunityMy Life',
 ' Just ListenCalendarEventsHonolulu PrideActivitiesAbout UsAnchors & Re

In [34]:
import re
from string import punctuation

def repreprocessing(raw_sentence):
    if isinstance(raw_sentence, float):
        # 부동 소수점일 경우 처리할 내용 추가
        return ""

    preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii", "ignore")
    # 구두점 제거와 동시에 소문자화
    return "".join(word for word in preproceseed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline if not isinstance(x, float)]
preprocessed_headline[:5]

['skip to contenthi now dailycontestsjoin our teamadvertise with uslivenewsmaui wildfiresweathersunrisesportspodcastshi now dailyentertainmentactivitiesabout ushomewatch livenewslocal headlinesnational newshnn investigatesthis is nowsubmit your photoslatest newscastsweatherlatest forecastshnn hurricane centersurf reporttsunami centertrafficsunriseas seen on sunrisehealthier hawaiiwhats trendinghowards business reportsmart money mondaycelebrating filam historyhawaiian word of the dayspecial reportsspotlight nowpodcastshawaii news nowisland beatmuthaship with stephanie lumthe other side of paradise with lynn kawanohnn overtimehnn offairtell me a story with jim mendozasportssuper bowl lviiibracket busterswarrior nationscoringlivestats  predictionshow to watchentertainmentna hoku hanohano awardskeiki hulacheap eatspacific pulsekupuna powerk5 meleteen talkcontestshi now dailyhi now japantalk storycommunitymy life',
 ' just listencalendareventshonolulu prideactivitiesabout usanchors  reporte

In [35]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) + 1
print("단어 집합의 크기 : %d" % vocab_size)

단어 집합의 크기 : 452


In [36]:
sequences = list()

for sentence in preprocessed_headline:
    print(sentence)
    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sentence])[0] 
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

skip to contenthi now dailycontestsjoin our teamadvertise with uslivenewsmaui wildfiresweathersunrisesportspodcastshi now dailyentertainmentactivitiesabout ushomewatch livenewslocal headlinesnational newshnn investigatesthis is nowsubmit your photoslatest newscastsweatherlatest forecastshnn hurricane centersurf reporttsunami centertrafficsunriseas seen on sunrisehealthier hawaiiwhats trendinghowards business reportsmart money mondaycelebrating filam historyhawaiian word of the dayspecial reportsspotlight nowpodcastshawaii news nowisland beatmuthaship with stephanie lumthe other side of paradise with lynn kawanohnn overtimehnn offairtell me a story with jim mendozasportssuper bowl lviiibracket busterswarrior nationscoringlivestats  predictionshow to watchentertainmentna hoku hanohano awardskeiki hulacheap eatspacific pulsekupuna powerk5 meleteen talkcontestshi now dailyhi now japantalk storycommunitymy life
 just listencalendareventshonolulu prideactivitiesabout usanchors  reportersnews

[[89, 3],
 [89, 3, 90],
 [89, 3, 90, 18],
 [89, 3, 90, 18, 91],
 [89, 3, 90, 18, 91, 35],
 [89, 3, 90, 18, 91, 35, 92],
 [89, 3, 90, 18, 91, 35, 92, 11],
 [89, 3, 90, 18, 91, 35, 92, 11, 93],
 [89, 3, 90, 18, 91, 35, 92, 11, 93, 94],
 [89, 3, 90, 18, 91, 35, 92, 11, 93, 94, 18],
 [89, 3, 90, 18, 91, 35, 92, 11, 93, 94, 18, 95]]

In [37]:
index_to_word = {}
for key, value in tokenizer.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print("빈도수 상위 582번 단어 : {}".format(index_to_word[5]))

빈도수 상위 582번 단어 : and


In [38]:
max_len = max(len(l) for l in sequences)
print("샘플의 최대 길이 : {}".format(max_len))

샘플의 최대 길이 : 125


In [39]:
sequences = pad_sequences(sequences, maxlen=max_len, padding="pre")
print(sequences[:3])

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0 89  3]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0 89  3 90]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  

In [40]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

print(X[:3])

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0 89]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0 89  3]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0 

In [41]:
print(y[:3])

[ 3 90 18]


In [42]:
y = to_categorical(y, num_classes=vocab_size)

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GRU

In [44]:
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200


23/23 - 2s - 83ms/step - accuracy: 0.0337 - loss: 6.1109
Epoch 2/200
23/23 - 1s - 40ms/step - accuracy: 0.0435 - loss: 6.0228
Epoch 3/200
23/23 - 1s - 40ms/step - accuracy: 0.0435 - loss: 5.8293
Epoch 4/200
23/23 - 1s - 41ms/step - accuracy: 0.0435 - loss: 5.7654
Epoch 5/200
23/23 - 1s - 40ms/step - accuracy: 0.0435 - loss: 5.7242
Epoch 6/200
23/23 - 1s - 41ms/step - accuracy: 0.0435 - loss: 5.6892
Epoch 7/200
23/23 - 1s - 41ms/step - accuracy: 0.0435 - loss: 5.6517
Epoch 8/200
23/23 - 1s - 40ms/step - accuracy: 0.0463 - loss: 5.5991
Epoch 9/200
23/23 - 1s - 40ms/step - accuracy: 0.0449 - loss: 5.5548
Epoch 10/200
23/23 - 1s - 41ms/step - accuracy: 0.0463 - loss: 5.4876
Epoch 11/200
23/23 - 1s - 40ms/step - accuracy: 0.0478 - loss: 5.3917
Epoch 12/200
23/23 - 1s - 41ms/step - accuracy: 0.0478 - loss: 5.2850
Epoch 13/200
23/23 - 1s - 39ms/step - accuracy: 0.0449 - loss: 5.1934
Epoch 14/200
23/23 - 1s - 41ms/step - accuracy: 0.0548 - loss: 5.1045
Epoch 15/200
23/23 - 1s - 39ms/step - acc

### GRU model save

In [45]:
model.save("./model/GRU.keras")

In [46]:
import tensorflow as tf
model = tf.keras.models.load_model("./model/GRU.keras")

In [47]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding="pre")

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [48]:
print(sentence_generation(model, tokenizer, "I am Kimminsup.", 35))

I am Kimminsup. did settlement from hpd arrested for gun been advantage of hawaiis climate to test and grow herbicideresistant corn seed production company may be production for a shoreline drag strip and the state has lost the
